<a href="https://colab.research.google.com/github/mgnarag/binarization-lowres/blob/main/200From_Arki_model5_red.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/From Arki/"

 313490484_646790150357239_663796780442369775_n.jpg
 361079911_248325231393045_2145216296096417082_n.jpg
'Applied Physics 184 FX-2'
 Architectural_designs
'BS Applied Physics'
 Classroom
'Colab Notebooks'
'CONSENT_NARAG_MARK JEREMY_G.pdf'
 CONSENT_NARAG_MARKJEREMY_G.pdf
'Dorm bill.png'
'Getting started.pdf'
 IMG_6546.PNG
'IMG_9359 (1).PNG'
 IMG_9359.PNG
'Information Sharing Consent Form - NARAG.pdf'
'Information Sharing Consent Form.pdf'
'[Journal of Cultural Heritage] Discovering artistic influences of painters'
'MS PHYSICS'
'MS Thesis videos'
'NARAG, MARK JEREMY, GACIAS.gdoc'
'NARAG, MARK JEREMY, GACIAS.pdf'
'NARAG, MARK JEREMY.jpg'
 NIP_Narag_MarkJeremy_Consent.pdf
 NIP_Narag_MarkJeremy_Informal.jpg
 NIP_Narag_MarkJeremy_Sablay.jpg
'PEHA 2021 Consent Form (fillable).pdf'
'PHOTO_NARAG_MARK JEREMY_G.jpg'
 PHOTO_NARAG_MARKJEREMY_G.jpg
'Physics 265'
'Physics 301 2S AY2122'
'Physics 305 Data Driven Astronomy'
'Research files'
'Screen Shot 2022-08-31 at 10.46.42.png'
'Screen Shot 2022-09-

In [3]:

import time
import pandas as pd


import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


from PIL import Image, ImageOps
import os
import numpy as np

epoch = 100
model_number = 5
size = 128

def crop(im):
    width, height = im.size
    data = []

    for j in np.arange(0,int(height/size)+1,0.66):
        for i in np.arange(0,int(width/size)+1,0.66):
            im1 = im.crop((0 + (size*i), 0 + (size*j), size + (size*i), size + (size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)


    return data

def crop_(im):
    width, height = im.size
    data = []

    for j in np.arange(0,int(height/size)+1,0.66):
        for i in np.arange(0,int(width/size)+1,0.66):
            im1 = im.crop((0 + (size*i), 0 + (size*j), size + (size*i), size + (size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            #im1 = im1/4
            data.append(im1)


    return data



input = np.array(crop_(ImageOps.grayscale(Image.open(file_path + "200red_Rizal_input.png"))))
print("Done reading the input of size = ", input.shape)

output = np.array(crop(ImageOps.grayscale(Image.open(file_path + "200Rizal_output.png"))))
print("Done reading the output of size = ", input.shape)

train_input = []
test_input = []
for i in range(0,len(input)):
    if i % 4 == 0:
        test_input.append(input[i])
    else:
        train_input.append(input[i])

train_input = np.array(train_input)
test_input = np.array(test_input)

print("Done reading the train input of size = ", train_input.shape)
print("Done reading the test input of size = ", test_input.shape)

train_output = []
test_output = []
for i in range(0,len(output)):
    if i % 4 == 0:
        test_output.append(output[i])
    else:
        train_output.append(output[i])

train_output = np.array(train_output)
test_output = np.array(test_output)

print("Done reading the train output of size = ", train_input.shape)
print("Done reading the test output of size = ", test_input.shape)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (115430400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Done reading the input of size =  (16632, 128, 128)
Done reading the output of size =  (16632, 128, 128)
Done reading the train input of size =  (12474, 128, 128)
Done reading the test input of size =  (4158, 128, 128)
Done reading the train output of size =  (12474, 128, 128)
Done reading the test output of size =  (4158, 128, 128)


In [4]:

n = 10
plt.figure(figsize=(10, 4))
for i in range(0,10):

    ax = plt.subplot(2, n, i + 1)
    plt.title("input")
    plt.imshow((test_input[i+1000]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    bx = plt.subplot(2, n, i + n + 1)
    plt.title("output")
    plt.imshow((test_output[i+1000]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()



In [5]:
start = time.time()
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

class autoenc(Model):
  def __init__(self):
    super(autoenc, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(size, size, 1)),
      layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(256, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(256, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(128, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(64, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = autoenc()


autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

model = autoencoder.fit(train_input, train_output,
                epochs=epoch,
                shuffle=True,
                validation_data=(test_input, test_output))

autoencoder.encoder.summary()


autoencoder.save(file_path + '1 Models/' + 'autoencoder_' + str(model_number)+'_red')

loss = model.history['loss']
loss = pd.DataFrame(loss)
loss.to_csv(file_path + '2 Loss/'+'loss_'+ str(model_number) + '_red.csv')
val_loss = model.history['val_loss']
val_loss = pd.DataFrame(val_loss)
val_loss.to_csv(file_path + '3 Validation loss/' +'val_loss_'+ str(model_number) + '_red.csv')

# record end time
end = time.time()

print("The time of execution of above program is :",
      (end-start), "seconds")


Epoch 1/100
390/390 [==============================] - 53s 102ms/step - loss: 0.0683 - val_loss: 0.0631
Epoch 2/100
390/390 [==============================] - 38s 97ms/step - loss: 0.0649 - val_loss: 0.0622
Epoch 3/100
390/390 [==============================] - 37s 96ms/step - loss: 0.0635 - val_loss: 0.1064
Epoch 4/100
390/390 [==============================] - 38s 96ms/step - loss: 0.0629 - val_loss: 0.0605
Epoch 5/100
390/390 [==============================] - 37s 95ms/step - loss: 0.0619 - val_loss: 0.0598
Epoch 6/100
390/390 [==============================] - 37s 95ms/step - loss: 0.0608 - val_loss: 0.0593
Epoch 7/100
390/390 [==============================] - 37s 95ms/step - loss: 0.0602 - val_loss: 0.0588
Epoch 8/100
390/390 [==============================] - 38s 97ms/step - loss: 0.0596 - val_loss: 0.0584
Epoch 9/100
390/390 [==============================] - 37s 95ms/step - loss: 0.0592 - val_loss: 0.0581
Epoch 10/100
390/390 [==============================] - 38s 97ms/step - 